In [8]:
# 01_create_embeddings_for_kaggle.ipynb

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import numpy as np
from tqdm import tqdm
import gc
import os

# --- 설정 ---
# Hugging Face에서 사용할 임베딩 모델 이름
MODEL_NAME = "BAAI/bge-large-en-v1.5" 

# 캐글 Competition 데이터 경로
# 이 경로는 캐글에서 데이터를 추가했을 때의 기본 경로입니다.
# COMPETITION_DATA_PATH = "/kaggle/input/jigsaw-agile-community-rules"
# TRAIN_DATA_PATH = os.path.join(COMPETITION_DATA_PATH, "train.csv")
# TEST_DATA_PATH = os.path.join(COMPETITION_DATA_PATH, "test.csv")
TRAIN_DATA_PATH = "train.csv"
TEST_DATA_PATH = "test.csv"

# 캐글 노트북의 출력 폴더 경로
# 이 경로에 저장된 파일만 데이터셋으로 만들 수 있습니다.
OUTPUT_DIR = "working" 
EMBEDDINGS_SAVE_PATH = os.path.join(OUTPUT_DIR, "reference_embeddings.npy")
LABELS_SAVE_PATH = os.path.join(OUTPUT_DIR, "reference_labels.npy")



In [2]:
# --- 1. 환경 설정 및 모델 로딩 ---
# (이전 코드와 동일)
print(">>> 1. 환경 설정 및 모델 로딩 시작")
torch.backends.cudnn.benchmark = True
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f"'{MODEL_NAME}' 모델을 다운로드하고 로딩합니다...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)
print(f"모델이 {device}에 로딩되었습니다.")

if torch.cuda.is_available():
    base_model = base_model.half()
    print("GPU 사용이 가능하여 모델을 half-precision(float16)으로 변환했습니다.")
print(">>> 모델 로딩 완료!\n")


# --- 2. 데이터 로딩 ---
print(">>> 2. 데이터 로딩 시작")
train = pd.read_csv(TRAIN_DATA_PATH)
test = pd.read_csv(TEST_DATA_PATH)
print("train.csv와 test.csv 파일을 성공적으로 불러왔습니다.")

MAX_TRAIN_SAMPLES = 10000 
if len(train) > MAX_TRAIN_SAMPLES:
    train = train.sample(n=MAX_TRAIN_SAMPLES, random_state=42)
    print(f"훈련 데이터가 너무 커서 {MAX_TRAIN_SAMPLES}개로 샘플링되었습니다.")
print(">>> 데이터 로딩 완료!\n")


# --- 3. 헬퍼 함수 정의 ---
# (get_embeddings_optimized, create_smart_reference_texts 함수는 이전 코드와 동일)
def get_embeddings_optimized(texts, model, tokenizer, device, batch_size=16, max_length=512):
    """주어진 텍스트 목록을 임베딩 벡터로 변환하는 함수 (최적화 버전)"""
    embeddings = []
    model.eval()
    
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="임베딩 생성 중"):
            batch_texts = texts[i:i+batch_size]
            inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)
            batch_embeddings = batch_embeddings.float().cpu().numpy()
            embeddings.append(batch_embeddings)
            if torch.cuda.is_available(): torch.cuda.empty_cache()
    
    return np.vstack(embeddings)

def create_smart_reference_texts(train_df, test_df, max_per_category=1000):
    """예측의 기준이 될 '참조 텍스트'와 라벨을 생성하는 함수"""
    reference_texts, reference_labels = [], []
    print("훈련 데이터로부터 참조 텍스트를 생성합니다...")
    violation_samples = train_df[train_df['rule_violation'] == 1].sample(min(len(train_df[train_df['rule_violation'] == 1]), max_per_category), random_state=42)
    non_violation_samples = train_df[train_df['rule_violation'] == 0].sample(min(len(train_df[train_df['rule_violation'] == 0]), max_per_category), random_state=42)
    balanced_train = pd.concat([violation_samples, non_violation_samples])
    for _, row in balanced_train.iterrows():
        reference_texts.append(f"r/{row['subreddit']}\nRule: {row['rule']}\nComment: {row['body']}")
        reference_labels.append(row['rule_violation'])
    print(f"훈련 데이터에서 {len(reference_texts)}개의 샘플을 추가했습니다.")
    sample_test = test_df.sample(min(len(test_df), 500), random_state=42)
    for _, row in sample_test.iterrows():
        rule_text = f"r/{row['subreddit']}\nRule: {row['rule']}\n"
        for i in [1, 2]:
            if pd.notna(row.get(f'positive_example_{i}')):
                reference_texts.append(rule_text + f"Comment: {row[f'positive_example_{i}']}")
                reference_labels.append(1)
        for i in [1, 2]:
            if pd.notna(row.get(f'negative_example_{i}')):
                reference_texts.append(rule_text + f"Comment: {row[f'negative_example_{i}']}")
                reference_labels.append(0)
    print(f"테스트 데이터의 예시를 추가하여 총 {len(reference_texts)}개의 참조 텍스트를 생성했습니다.")
    return reference_texts, reference_labels

# --- 4. 메인 실행 로직 ---
print(">>> 4. 참조 임베딩 생성 시작")
reference_texts, reference_labels = create_smart_reference_texts(train, test)
reference_embeddings = get_embeddings_optimized(reference_texts, base_model, tokenizer, device)



>>> 1. 환경 설정 및 모델 로딩 시작
'BAAI/bge-large-en-v1.5' 모델을 다운로드하고 로딩합니다...


모델이 cuda에 로딩되었습니다.
GPU 사용이 가능하여 모델을 half-precision(float16)으로 변환했습니다.
>>> 모델 로딩 완료!

>>> 2. 데이터 로딩 시작
train.csv와 test.csv 파일을 성공적으로 불러왔습니다.
>>> 데이터 로딩 완료!

>>> 4. 참조 임베딩 생성 시작
훈련 데이터로부터 참조 텍스트를 생성합니다...
훈련 데이터에서 1998개의 샘플을 추가했습니다.
테스트 데이터의 예시를 추가하여 총 2038개의 참조 텍스트를 생성했습니다.


임베딩 생성 중: 100%|██████████| 128/128 [00:13<00:00,  9.21it/s]


In [9]:
# 1. 저장하기 전에 출력 폴더가 있는지 확인하고, 없으면 생성합니다.
os.makedirs(OUTPUT_DIR, exist_ok=True) 
print(f"생성된 임베딩과 라벨을 '{OUTPUT_DIR}' 폴더에 저장합니다...")

# 2. 넘파이(Numpy) 배열을 바이너리 파일(.npy) 형태로 저장합니다.
np.save(EMBEDDINGS_SAVE_PATH, reference_embeddings)
np.save(LABELS_SAVE_PATH, np.array(reference_labels))

# 3. 로컬 환경에 맞는 완료 메시지를 출력합니다.
print(">>> 모든 작업 완료! ✨")
print(f"'{EMBEDDINGS_SAVE_PATH}' 와 '{LABELS_SAVE_PATH}' 파일이 성공적으로 저장되었습니다.")


생성된 임베딩과 라벨을 'working' 폴더에 저장합니다...
>>> 모든 작업 완료! ✨
'working/reference_embeddings.npy' 와 'working/reference_labels.npy' 파일이 성공적으로 저장되었습니다.
